<a href="https://colab.research.google.com/github/B-Lorentz/riverine/blob/master/py/sph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from jax import grad, vmap, jit, lax
from jax import numpy as jnp
from jax import random as rand
import jax
import functools as func
from dataclasses import dataclass

In [ ]:
import pathlib
def once_in_a_VM(fname):
  pat = pathlib.Path(fname)
  if pat.exists():
      return False
  else:
    pat.mkdir()
    return True
if once_in_a_VM("download"):
  !git clone https://github.com/B-Lorentz/riverine.git

In [ ]:
if once_in_a_VM("install"):
  !pip install -e riverine/py
  !pip install --upgrade matplotlib

In [ ]:
from utils.height_field import *

In [ ]:

def show_terrain(b, d, threshold=1e-7):
  #print(d.shape, b.shape)
  fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(14, 10))
  x, y = np.meshgrid(np.arange(b.shape[1]), np.arange(b.shape[0]))
  #print(x.shape, y.shape)
  color = np.array([0.0,0.0,0.9])*((d>threshold)*(0.1 + (b+d)/(b+d).max()))[:,:,np.newaxis] + np.array([0.8, 0.5, 0])*(b/b.max()*(d<threshold))[:,:,np.newaxis]

  ax.plot_wireframe(x, y, b, color="black")
  surf = ax.plot_surface(x, y, b+d, facecolors=color, antialiased=False, shade=True)
 
  return ax

h = tilt_hf(20, 0.1, 0.0)[:,:10]
w = np.zeros_like(h)+.0
w[4, 6] = 0.01 
w[4, 7] = 0.01 
ax = show_terrain(h, w)
ax.view_init(30, -120)

An eulerian 2D method using shallow-water equations:
Mei, Decaudin & Hu:
*Fast Hydraulic Erosion Simulation and Visualization on GPU*

In [ ]:
dirs = np.array([[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1]])
#dirs = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
def flow_init(h, dirs):
  return np.zeros((dirs.shape[0], *h.shape))

def flow_conv(h, dirs):
  kernel = np.zeros((dirs.shape[0], 1, 3, 3))
  for i, dir in enumerate(dirs):
    d = np.sqrt(np.power(dir, 2).sum())
    kernel[i, 0, 1, 1] += 1/d
    kernel[i, 0, 1+dir[0], 1+dir[1]] += -1/d
 
  res = lax.conv(h[np.newaxis,np.newaxis,:,:], kernel, (1,1),padding="VALID").squeeze()
 
  return jnp.pad(res, ((0,0), (1, 1), (1, 1)), mode="edge")

def collect_conv(f, dirs):
  kernel = np.zeros((1, dirs.shape[0], 3, 3))
  for i, dir in enumerate(dirs):
    kernel[0, i, 1, 1] += -1.0
    kernel[0, i, 1-dir[0], 1-dir[1]] += 1.0
  
  res = lax.conv(jnp.pad(f, ((0,0),(1, 1), (1, 1)), mode="edge")[np.newaxis,:,:,:], kernel, (1,1),padding="VALID", precision=lax.Precision.HIGHEST).squeeze()
  return res

def f_scale(f, w, dt):
  f = jnp.maximum(f, 0.0)
  return f*jnp.minimum(1.0, w/(dt * (f.sum(0) + 1e-9)))

def flow_quiver(h, w, f, markers = ([], [])):
  plt.figure(figsize=(18, 10))
  x, y = np.meshgrid(np.arange(h.shape[-2]), np.arange(h.shape[-1]))
 
  plt.imshow(h.T, origin="lower", )
  plt.scatter(x.T, y.T, s=3000*w, c="r")
  plt.scatter(markers[0], markers[1], s=100, marker="x" )
  plt.gca().xaxis.set_major_locator(MultipleLocator(1))
  plt.gca().yaxis.set_major_locator(MultipleLocator(1))
  plt.grid()
  print(h.shape)
  for dir, f_ in zip(dirs, f):
    d = np.sqrt(np.power(dir, 2).sum())
    plt.quiver(x.T, y.T, dir[0]*f_/d, dir[1]*f_/d, scale=1, units="xy", width=0.04, headlength=5)
 
 
f = flow_init(h, dirs)
f[1] = 1.0
f = flow_conv(h + w, dirs)
#print(f[2])
print(f.sum())
f = f_scale(f, w, 0.1)
print(f.sum())
w1 = collect_conv(f, dirs)
print(w1.min(), w1.max())
flow_quiver(h, 10*(w+w1), f*300)
print(w.sum(), w1.sum())

In [ ]:
def v_conv(f, dirs):
  kernel = np.zeros((2, dirs.shape[0], 3, 3))
  bvs = np.array(((1, 0), (0, 1.0)))
  for i, dir in enumerate(dirs):
    dot = (bvs*dir).sum(-1)/(dir**2).sum()
   
    kernel[:, i, 1, 1] += 0.5*dot
    kernel[:, i, 1+dir[0], 1+dir[1]] += 0.5*dot
  res = lax.conv(jnp.pad(f, ((0, 0), (1, 1), (1, 1)), mode="edge")[np.newaxis,:,:], kernel, (1,1),padding="VALID").squeeze()
  return res

def vector_quiver(vs, **kwargs):
  x, y = np.meshgrid(np.arange(vs.shape[-2]), np.arange(vs.shape[-1]))
  plt.quiver(x.T, y.T, vs[0], vs[1], **kwargs)
ftest = flow_init(h, dirs)
ftest[0,:,:] = 0.1
ftest[6,:,:] = 0.3
vs = v_conv(ftest, dirs)
flow_quiver(h, 0, 4*ftest)
vector_quiver(4*vs, scale=1, units="xy", width=0.03, headlength=5, color="blue")

In [ ]:
def make_erosion( K_dis, K_depo, signed_capacity, woff=1.0, wc=-1 ):
  def erosion(h, w, s, f):
    wv = v_conv(f, dirs)*(w + woff)**wc
    vabs = jnp.sqrt((wv**2).sum(0))
  
    C_s = signed_capacity(vabs, s)
    erodes = (C_s>0)
    return (erodes*K_dis + (~erodes)*K_depo )*(C_s)
  return erosion

def make_update(source, dt, dirs, gA, nu, K_evap, erosion, logger=lambda h, w, f, s, dh, dw: None):
  l = np.sqrt(np.power(dirs, 2).sum(-1))
  def update(h, w, f, s):
    # Source term
    w = w + source*dt
    # Flow update

    df = gA*flow_conv(h+w, dirs) - nu*f
    f = f + dt*df
    f = f_scale(f, w, dt)
    dw = collect_conv(f, dirs)*dt
    w = w + dw
    # Sediment carry
    
    dh = erosion(h, w, s, f)*dt
    h = h - dh
    s = s + dh
    
    # Sediment transport
    fs = f_scale(f*s, s, dt)
    s = s + collect_conv(fs, dirs)*dt
    s = jnp.maximum(s, 0)

    # Evaporation
    w = w*(1- K_evap*dt)

    return h, w, f, s, logger(h, w, f, s, dh, dw)
  return update

In [ ]:

def zinit(h, dirs):
  return  np.zeros_like(h), flow_init(h, dirs)
def trace_sim(update, h, w, N):
  s, f = zinit(h, dirs)
  
  def upd(state, _):
    h, w, f, s, log= update(*state)
    return (h, w, f, s), log
  end, log = jax.lax.scan(jit(upd), (h, w, f, s), xs=None, length=N)
  return end, log 

for_sim_ = lambda upd, N : lambda state : jax.lax.fori_loop(0, N, jit(upd), state)

def for_sim(update, h, w, N):
  s, f = zinit(h, dirs)
  
  def upd(i, state):
    h, w, f, s, log= update(*state)
    return (h, w, f, s)
  end = for_sim_(upd, N)((h,w, f, s))
  return end

In [ ]:
zero_erosion = make_erosion( K_dis=0.0, K_depo=0.0, signed_capacity=lambda v,s: 0.0)
res, log = trace_sim(make_update(0, 0.05, dirs, 9.81, 0.1, 
                                        K_evap=0.01, erosion=zero_erosion, logger=lambda h, w, f, s, dh, dw: w), h, w, 100)

In [ ]:
ax = show_terrain(h, log[200].to_py())
ax.view_init(30, -120)

In [ ]:
plt.plot(log.sum(-1).sum(-1))
plt.grid()

In [ ]:
def basin_experiment():
  N = 100
  h1 = basin_hf(N)
  w1 = np.maximum(tilt_hf(N, 0.1, 0.1)+0.2-h1,0)

 
  res, log = trace_sim(make_update(0, 0.1, dirs, gA=9.81, nu=0.1,K_evap=0.01, 
                                        erosion=zero_erosion,
                                        logger=lambda h, w, f, s, dh, dw: w), h1, w1, 1000)

  
  return h1, w1, res, log

def basin_plot(result):
  h1, w1, res, log = result

  ax = show_terrain(h1, w1)
  ax.view_init(20, 120)
  ax = show_terrain(h1, log[-1].to_py())
  ax.view_init(20, 120)
  
  plt.figure(figsize=(8, 4))
  plt.plot(log[:, 70, 70])
  plt.grid()
 
  plt.xlabel("t", fontsize=14)
  plt.ylabel("water level", fontsize=14)

In [ ]:
res = basin_experiment()

In [ ]:
basin_plot(res)

In [ ]:
from matplotlib import colors
def plot_v(f, name, **kwargs):
  plt.imshow(f.T, origin="lower", **kwargs)
  plt.title(name, fontsize=14)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.colorbar()


def before_after(res):
  
  h, w, source, (h1, w1, f, s) = res
  N = h1.shape[0]

  def plot_marginals(w):
    norm = lambda q: q/np.abs(q).max()*10
    plt.plot(norm(w.sum(0)), np.arange(N), "r")
    plt.plot(norm(w.sum(-1)), "r")

  plt.figure(figsize=(18, 18))
  

  plt.subplot(221)
  v = v_conv(f, dirs)
  print(v.shape)
  vector_quiver(1.0*v, color="white", units="xy", scale=0.5, headlength=1)
  vabs = np.sqrt((v**2).sum(0))
  plot_v(vabs, "|v|*w")
  plot_marginals(vabs)

  plt.subplot(222)
  plot_v(w1, r"$w_1$: water level")
  plot_marginals(w1)

  plt.subplot(223)
  plot_v(h1, r"$h_1 + w1$: height field ", cmap="terrain", vmin = -0.5)
  plt.plot(10*h1[0,:],np.arange(N), "r")
  plt.plot(10*(h1[0,:]+w1[0,:]),np.arange(N), "b")

  plt.subplot(224)
  dh = h-h1
  plot_v(dh, r"-$\Delta h$: height loss",norm=colors.CenteredNorm(), cmap="coolwarm")
  
  plot_marginals(dh)


## Csatorna kisérletek

In [ ]:
def wavy_field(N, amplitude, tilt, water_level):
  h = sine_hf(N, 1.0, amplitude)
  #h[-5:,:] += 1.0
  w = np.zeros((N, N))
  w += np.maximum(water_level-h, 0)
  return h+tilt_hf(N, 0.0, tilt)+2, w
def wavy_channel_experiment(wf, erosion):
  h, w = wf
  source = np.zeros(w.shape)
  #source[-1,:] += 1.0*(w[-1,:]>0.001)*w[-1,:]
  res = for_sim(make_update(source, 0.1, dirs, gA=9.81, nu=0.1, K_evap=0.001, 
                                        erosion=erosion), h, w, 5000)
  return h, w, source, res

wf = wavy_field(100, 0.0, 0.5, 0.5)
ax = show_terrain(wf[0], wf[1])

### Egyenes csatorna

Csináltunk egy egyszerű, lejtő csatornát. Először erózió nélkül megnézzük, milyen egyensúlyi sebességeloszlás alakul ki.

In [ ]:
res = wavy_channel_experiment(wf, zero_erosion)
before_after(res)

In [ ]:
erosion = make_erosion( K_dis = 0.1, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, wc=0)
res = wavy_channel_experiment(wf, erosion)
before_after(res)

Ez a víz többet tud szállítani

In [ ]:
erosion = make_erosion( K_dis = 0.1, K_depo = 0.01, signed_capacity=lambda v, s: 1.0*v - s, wc=0)
res = wavy_channel_experiment(wf, erosion)
before_after(res)

Ha a szállítási kapacítás a sebességgel és nem az impluzussal arányos, egy széles és visznylag lapos medret kapunk

In [ ]:
erosion = make_erosion( K_dis = 0.1, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, woff=0.05,wc=-1, )
res = wavy_channel_experiment(wf, erosion)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py())

### Hullámos csatorna

In [ ]:
wf = wavy_field(100, 0.4, 0.5, 0.5)
ax = show_terrain(wf[0], wf[1])

Megfigyelhető, hogy a hidrodinamikai modellünk a valósággal ellentétben a belső íveken jelez gyorsabb folyást:

In [ ]:
res = wavy_channel_experiment(wf, zero_erosion)
before_after(res)

In [ ]:
res = wavy_channel_experiment(wf, make_erosion( K_dis = 0.1, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, wc=0))
before_after(res)

In [ ]:
res = wavy_channel_experiment(wf, make_erosion( K_dis = 0.1, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, woff=0.05,wc=-1, ))
before_after(res)

## Hegy-síkság kisérletek

In [ ]:
def plain_field(N, a, d, l):
  x, y, _,= cors(N)
  h = np.exp(a*(x-d))
  h1 = np.copy(h)
  w = np.zeros((N, N))
  h1[45:55, -l:] = h1[45:55, -l:].min()
  w += h-h1
  return h1.T+tilt_hf(N, 0.0, 0.1)+1.0, w.T
def plain_experiment(wf, erosion, nu, bumps=0, K_evap=0.001, N=5000):
  if len(wf) == 2:
    h, w = wf
    source = np.zeros(w.shape)
  else:
    h, w, source = wf
  bp = bumps*(w<1e-7)
  res = for_sim(make_update(source, 0.1, dirs, gA=9.81, nu=nu, K_evap=K_evap, 
                                        erosion=erosion), h+bp, w, N=N)
  return h+bp, w, source, res

wf = plain_field(100, 0.05, 100, 10)
ax = show_terrain(wf[0], wf[1])

### Különböző mértékű disszipáció

In [ ]:
res = plain_experiment(wf, zero_erosion, nu=0.01)
before_after(res)

In [ ]:
res = plain_experiment(wf, zero_erosion, nu=0.1)
before_after(res)

In [ ]:
res = plain_experiment(wf, zero_erosion, nu=1.0)
before_after(res)

### Különböző mértékő erózió

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 0.01, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, woff=0.05,wc=0, ), nu=0.1)
before_after(res)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 0.001, K_depo = 0.01, signed_capacity=lambda v, s: 0.1*v - s, woff=0.05,wc=0, ), nu=0.1)
before_after(res)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 0.005, K_depo = 0.001, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=0, ), nu=0.1)
before_after(res)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 0.005, K_depo = 0.001, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=0, ), nu=0.01)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

### v-arányos erózió

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.3)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.03)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-2, K_depo = 2e-3, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.03)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

## Véletlen kezdetű hegy-síkság kisérletek

In [ ]:
def bumps_hf(N, a, maxf, key):
  f = rand.poisson(rand.PRNGKey(key), a, shape=(N, N))
  f_ = np.fft.fftshift(np.fft.fft2(f), )
  x, y, _ = cors(N)
  f_[(x-N)**2 + (y-N)**2 > maxf**2] *= 0 
  f2 = np.abs(np.fft.ifft2(np.fft.fftshift(f_)))
  return f2
bp = bumps_hf(100, 10, 45.0, 137)
plt.figure(figsize=(8, 8))
plt.imshow(bp)
print(bp.min(), bp.max())


In [ ]:
bp = bumps_hf(100, 3, 20.0, 137)
print(bp.min(), bp.max())
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.03, 
                       bumps=bp*0.01)
before_after(res)

In [ ]:
bp = bumps_hf(100, 10, 35.0, 137)
print(bp.min(), bp.max())
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.01,wc=-1.0, ), nu=0.03, 
                       bumps=bp*0.04+0.1)
before_after(res)

In [ ]:
show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

In [ ]:
bp = bumps_hf(100, 3, 10.0, 137)
print(bp.min(), bp.max())
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.03, 
                       bumps=bp*0.08+0.1)
before_after(res)

In [ ]:
show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

In [ ]:
import cv2
from matplotlib.cm import get_cmap
terra = get_cmap("terrain")
def Kth_log(update, h, w, N, K):

 def upd(i, state):
    h, w, f, s, log= update(*state)
    return (h, w, f, s)

 inner = for_sim_(upd, K)
 def logged(h, w, f, s):
   h, w, f, s = inner((h, w, f, s))
   return h, w, f, s, (h, w)
 res, log = trace_sim(logged, h, w, N)
 #res = for_sim(logged, h, w, N)
 return res, log

def render(h, w, mi, ma, shift, wl=0.03, wmax=1.0):
 
  h_ = h-mi
  h_ = h_/ma
  res = terra(h_+shift)
  wi = (w>wl)[:,:,np.newaxis]
  res *= ~wi
  res += np.clip(wi*np.stack((1.0-w/wmax, 1.0-w/wmax, np.ones_like(w), np.ones_like(w)), -1), 0.0, 1.0)
  return res[:,:,:3]

def test_vid(wf, erosion, nu, bumps, fname, u=1, K_evap=0.001, N=500, K=10, wmax=0.3, wl=0.03):
  if len(wf) == 2:
    h, w = wf
    source = np.zeros(w.shape)
  else:
    h, w, source = wf
  bp = bumps*(w<1e-7)
  res, log = Kth_log(make_update(source, 0.1, dirs, gA=9.81, nu=nu, K_evap=K_evap, 
                                        erosion=erosion), h+bp, w, N, K)
  print(log[0].shape)
  sh = (u*h.shape[0], u*h.shape[1])
  writer = cv2.VideoWriter(fname,cv2.VideoWriter_fourcc(*"XVID"), 30, sh)
  mi, ma, shift = h.min(), h.max()-h.min(), 0.35
  for step in zip(log[0].to_py(), log[1].to_py()):
    h_, w_ = step
    r = render(h_, w_, mi, ma, shift, wmax=wmax, wl=wl)*255
    res = cv2.resize(cv2.cvtColor( r.astype(np.uint8), cv2.COLOR_RGB2BGR ), sh , interpolation=0)
    
    writer.write(res)
  writer.release()

  return h+bp, w, source, res

res1= test_vid(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.05,wc=-1.0, ), nu=0.03, 
                       bumps=bp*0.08+0.1, fname="islands.avi", u=3)

## Vízesés-kisérlet

In [ ]:
def waterfall_field(N, tilt, height ,a, b):

  h = tilt_hf(N, tilt, 0.0)
  print(h.shape)
  h[:, N//2+5:] += height
  h[:, (N//2-5):N//2+5] = np.linspace(h[0, N//2-5],h[0, N//2+5], 10)
  h1 = np.copy(h)
  w = np.zeros((N, N))
  h1[N//2-a:N//2+a, :] += -b
  w += (h-h1)*0.9
  return h1.T+1.0, w.T
wf = waterfall_field(100, 0.3, 1.0, 10, 0.1)
ax = show_terrain(wf[0], wf[1])

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 0.4*v - s, woff=0.01,wc=-1, ), nu=0.3, 
                       bumps=0)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py(), res[3][1].to_py(), 0.03)

## Esős kisérletek

In [ ]:
def runoff_hf(l, rain):
  N = 400
  edge = -abs_hf(N).T*0.1
  #edge = np.zeros_like(x)#np.maximum(((x-N//2)/N*2)**2, ((y-N//2)/N*2)**2)
  #pl = edge<l
  #edge[pl] = edge[pl].max()
  
  edge += gauss_hf(N, 210,210, 70)*0.5
  edge += gauss_hf(N, 350,150, 30)*0.7
  edge += gauss_hf(N, 110,110, 40)*0.3
  edge += tilt_hf(N, 0.1, 0.0)
  edge += bumps_hf(N, 20, 10, 42)*0.1
  s = np.zeros_like(edge)
  s[5:-5, 5:-5] += rain
  return edge.T+0.1, np.zeros_like(edge).T, s.T

wf = runoff_hf(0.75, 0.001)
ax = show_terrain(wf[0], wf[1])
plt.figure(figsize=(6,6))
plt.imshow(wf[0])

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 0.1*v - s, woff=0.01,wc=-1, ), nu=0.03, 
                       bumps=0, K_evap=0.01, N=3000)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py()+1, res[3][1].to_py(), 0.01)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, signed_capacity=lambda v, s: 0.5*v - s, woff=0.01,wc=-1, ), nu=0.03, 
                       bumps=0, K_evap=0.01, N=3000)
before_after(res)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 3e-4, K_depo = 3e-4, signed_capacity=lambda v, s: 1.0*v - s, woff=0.01,wc=-1, ), nu=0.03, 
                       bumps=0, K_evap=0.02, N=3000)
before_after(res)

In [ ]:
res = plain_experiment(wf, make_erosion( K_dis = 4e-3, K_depo = 4e-3, signed_capacity=lambda v, s: 1.0*v - s, woff=0.01,wc=0, ), nu=0.01, 
                       bumps=0, K_evap=0.01, N=5000)
before_after(res)

In [ ]:
ax = show_terrain(res[3][0].to_py()[2:-2, 2:-2]+1, res[3][1].to_py()[2:-2, 2:-2], 0.01)

In [ ]:
res = test_vid(wf, make_erosion( K_dis = 2e-4, K_depo = 2e-4, 
                                signed_capacity=lambda v, s: 1.0*v - s, woff=0.01,wc=-1, ), nu=0.01, 
                       bumps=0, fname="runoff.avi",K_evap=0.02, N=500, K=10, u=1, wmax=0.5, wl=0.01)